## Wiki solutions

In [1]:
import requests
from bs4 import BeautifulSoup

In [8]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata
import re

cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Leipzig','Cologne','Dresden','Hannover','Paris', 'Barcelona','Lisbon','Madrid']
# cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

def Get_City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        latitudeHTML = soup.select_one('.latitude')
        ret_dict['lat'] = latitudeHTML.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict



list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city) #'https://en.wikipedia.org/wiki/Berlin'
    webSiteContentAsHTML = requests.get(url,'html.parser')
    soup = bs(webSiteContentAsHTML.content)
    cityInfo = Get_City_info(soup)
    list_of_city_info.append(cityInfo)
    
    
df_cities = pd.DataFrame(list_of_city_info)
# df_cities = df_cities.set_index('city')
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Michael Müller (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,664,088","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Hamburg,Peter Tschentscher (SPD),755.22 km2 (291.59 sq mi),NaN,"1,845,229","2,484,800[1]","5,107,429",53°33′55″N,10°00′05″E
2,Frankfurt,Peter Feldmann[1] (SPD),248.31 km2 (95.87 sq mi),112 m (367 ft),"764,104","2,319,029[3]","5,604,523[2]",50°7′N,8°41′E
3,Munich,NaN,310.71 km2 (119.97 sq mi),"520 m (1,710 ft)","1,488,202","2,606,021","5,991,144[2]",48°08′15″N,11°34′30″E
4,Stuttgart,NaN,207.33 km2 (80.05 sq mi),245 m (804 ft),"630,305","2,787,724 (31 Dec 2018)[3]","5,300,000 (2015)[2]",48°46′55″N,9°11′02″E
5,Leipzig,NaN,297.36 km2 (114.81 sq mi),NaN,"597,493",NaN,"1,001,220 (LUZ)[2]",51°20′N,12°23′E
6,Cologne,NaN,405.15 km2 (156.43 sq mi),37 m (121 ft),"1,083,498","3,500,000 (Cologne Bonn)","10,500,000 (Rhine-Ruhr)",50°56′11″N,6°57′10″E
7,Dresden,NaN,328.8 km2 (127.0 sq mi),113 m (371 ft),"556,227","790,400[3]","1,343,305[2]",51°03′00″N,13°44′24″E
8,Hanover,NaN,204.01 km2 (78.77 sq mi),55 m (180 ft),"534,049",NaN,"1,119,032",52°22′N,9°43′E
9,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,175,601","10,785,092","13,024,518",48°51′24″N,2°21′08″E


In [ ]:
url = 'https://www.bbc.com/news'
response = get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup

for tag in soup.select('li[data-entityid^="most-popular-read"]'):
    print(tag.get_text())

In [ ]:
url = 